In [24]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('application/'))))

In [25]:
from application.models import User, GoogleCalendarEvent, SlackConversationRead, SlackUserEvent
import datetime as dt
import pandas as pd
import numpy as np
import math
from sqlalchemy import any_

# basically, once you have a current user, you can do things with it
current_user = User.query.filter(User.slack_user != None, User.google_calendar_user != None).all()[-3]

WEEKDAY_OFFSET = 1 # configures start of week is Sunday

# this is functional programming: it returns a callable function
def convert_to_user_timezone_function(slack_timezone_offset):
  return lambda datetime: datetime + dt.timedelta(hours=slack_timezone_offset / 3600.0)

# this is functional programming: it returns a callable function
def convert_to_utc_timezone_function(slack_timezone_offset):
  return lambda datetime: datetime - dt.timedelta(hours=slack_timezone_offset / 3600.0)

def build_google_calendar_event_date_filter(start_utc_datetime, end_utc_datetime):
  start_between_filter = (GoogleCalendarEvent.start_time >= start_utc_datetime) & (GoogleCalendarEvent.start_time < end_utc_datetime)
  end_between_filter = (GoogleCalendarEvent.end_time >= start_utc_datetime) & (GoogleCalendarEvent.end_time < end_utc_datetime)
  encompass_filter = (GoogleCalendarEvent.start_time < start_utc_datetime) & (GoogleCalendarEvent.end_time > end_utc_datetime)
  return start_between_filter | end_between_filter | encompass_filter

def build_slack_user_event_date_filter(start_utc_datetime, end_utc_datetime):
  return (SlackUserEvent.event_datetime >= start_utc_datetime) & (SlackUserEvent.event_datetime < end_utc_datetime) \
    & (SlackUserEvent.slack_event_type == any_(['message', 'reaction_added', 'reaction_removed']))

def build_slack_conversation_read_date_filter(start_utc_datetime, end_utc_datetime):
  return (SlackConversationRead.period_start_datetime >= start_utc_datetime) & (SlackConversationRead.period_start_datetime < end_utc_datetime)

def create_time_series(start, end):
  x = pd.date_range(start=start,end=end, freq='5T')
  return x
  # x_axis = [pt.to_pydatetime() for pt in x.tolist()]
  # return dict.fromkeys(x_axis)

def rounddown_next_5min(event_datetime):
  event_datetime = pd.Timestamp(event_datetime)
  nsecs = event_datetime.minute*60 + event_datetime.second + event_datetime.microsecond*1e-6
  delta = -math.floor(nsecs / 300) * 300 + nsecs
  return event_datetime - dt.timedelta(seconds=delta)

vector_rounddown_next_5min = np.vectorize(rounddown_next_5min) # allows this function to be called on dataframe column

str_index_fnc = lambda real_datetime: str(real_datetime)
str_index_fnc_vec = np.vectorize(str_index_fnc)

user_tz_conversion_function = convert_to_user_timezone_function(current_user.slack_user.slack_timezone_offset)
utc_tz_conversion_function = convert_to_utc_timezone_function(current_user.slack_user.slack_timezone_offset)

current_user_local_time = user_tz_conversion_function(dt.datetime.utcnow())

# finding Sunday before current time
start_week_local_time = current_user_local_time - \
          dt.timedelta(days=current_user_local_time.weekday() + WEEKDAY_OFFSET)
start_week_local_time = dt.datetime(
                          year = start_week_local_time.year,
                          month = start_week_local_time.month,
                          day = start_week_local_time.day,
                          hour = 0,
                          minute = 0,
                          second = 0,
                          microsecond = 0)

# finding Saturday after current time
end_week_local_time = current_user_local_time + \
            dt.timedelta(days=(7 - current_user_local_time.weekday() - WEEKDAY_OFFSET))
end_week_local_time = dt.datetime(
                        year = end_week_local_time.year,
                        month = end_week_local_time.month,
                        day = end_week_local_time.day,
                        hour = 23,
                        minute = 59,
                        second = 59,
                        microsecond = 999999)

start_week_utc_time = utc_tz_conversion_function(start_week_local_time)
end_week_utc_time = utc_tz_conversion_function(end_week_local_time)

calendar_filter = \
  (GoogleCalendarEvent.google_calendar_user_id == current_user.google_calendar_user.id) \
  & build_google_calendar_event_date_filter(start_week_utc_time, end_week_utc_time)

slack_conversation_read_filter = \
  (SlackConversationRead.slack_user_id == current_user.slack_user.id) \
  & build_slack_conversation_read_date_filter(start_week_utc_time, end_week_utc_time)

slack_user_event_filter = \
  (SlackUserEvent.slack_user_id == current_user.slack_user.id) \
  & build_slack_user_event_date_filter(start_week_utc_time, end_week_utc_time)


'''
  The following can all be put into a function and given current_user and a start / end datetime. 
  It will then spit out a JSON of 
'''
datetime_index = pd.date_range(start=start_week_utc_time, end=end_week_utc_time, freq='5T')
activity_df = pd.DataFrame(datetime_index, index=datetime_index, columns=['datetime_utc'])

scr_df = pd.read_sql(SlackConversationRead.query.filter(slack_conversation_read_filter).statement, SlackConversationRead.query.session.bind)
scr_df['period_span'] = scr_df['period_end_datetime'] - scr_df['period_start_datetime']
# getting out some buggy data captures. NOT SURE OF THE CAUSE OF THESE:
scr_df = scr_df.loc[scr_df['period_span'] < dt.timedelta(minutes=6)]
scr_df['rounded_period_start_datetime'] = vector_rounddown_next_5min(scr_df['period_start_datetime'])
slack_conversation_read_series = scr_df.groupby(by=['rounded_period_start_datetime']).count()['id']
slack_conversation_read_series.name = 'slack_conversation_read_count'
activity_df['slack_conversation_read_count'] = slack_conversation_read_series

activity_df['google_calendar_event_id'] = np.nan
gce = GoogleCalendarEvent.query.filter(calendar_filter).all()
for calendar_event in gce:
  activity_df.loc[(activity_df.index >= calendar_event.start_time) & (activity_df.index < calendar_event.end_time), \
    ['google_calendar_event_id']] = calendar_event.id

sue_df = pd.read_sql(SlackUserEvent.query.filter(slack_user_event_filter).statement, SlackUserEvent.query.session.bind)
sue_df['rounded_event_datetime'] = vector_rounddown_next_5min(sue_df['event_datetime'])
slack_user_event_series = sue_df.groupby(by=['rounded_event_datetime']).count()['id']
activity_df['slack_user_event_count'] = slack_user_event_series

activity_df['slack_user_event_count'] = activity_df['slack_user_event_count'].fillna(0)
activity_df['slack_conversation_read_count'] = activity_df['slack_conversation_read_count'].fillna(0)
activity_df['in_meeting'] = activity_df['google_calendar_event_id'].notna().astype(int)

# Optionally, can datetime indexes to readable strings which need to be parsed
activity_df.index = str_index_fnc_vec(activity_df.index)


In [26]:
activity_df

,datetime_utc,slack_conversation_read_count,google_calendar_event_id,slack_user_event_count,in_meeting
2020-02-02 05:00:00,2020-02-02 05:00:00,0.0,NaN,0.0,0
2020-02-02 05:05:00,2020-02-02 05:05:00,0.0,NaN,0.0,0
2020-02-02 05:10:00,2020-02-02 05:10:00,0.0,NaN,0.0,0
2020-02-02 05:15:00,2020-02-02 05:15:00,0.0,NaN,0.0,0
2020-02-02 05:20:00,2020-02-02 05:20:00,0.0,NaN,0.0,0
...,...,...,...,...,...
2020-02-10 04:35:00,2020-02-10 04:35:00,0.0,NaN,0.0,0
2020-02-10 04:40:00,2020-02-10 04:40:00,0.0,NaN,0.0,0
2020-02-10 04:45:00,2020-02-10 04:45:00,0.0,NaN,0.0,0
2020-02-10 04:50:00,2020-02-10 04:50:00,0.0,NaN,0.0,0


In [43]:
from numpy.lib.stride_tricks import as_strided as stride
def roll(df, w, **kwargs):
    v = df.values
    d0, d1 = v.shape
    s0, s1 = v.strides

    a = stride(v, (d0 - (w - 1), w, d1), (s0, s0, s1))

    rolled_df = pd.concat({
        row: pd.DataFrame(values, columns=df.columns)
        for row, values in zip(df.index, a)
    })
    return rolled_df.groupby(level=0, **kwargs)

# then go roll(df_name, 3).apply(function_that_takes_in_small_df)

SLACK_CONVERSATION_READ_COLUMN_NAME = 'slack_conversation_read_count'
SLACK_USER_EVENT_COLUMN_NAME = 'slack_user_event_count'
IN_MEETING_COLUMN_NAME = 'in_meeting'

def period_interrupted(df, min_interrupt_len=None, \
    min_interrupt_read_amount=1, min_interrupt_send_amount=1):
  if not min_interrupt_len:
    min_interrupt_len = len(df)
  length = len(df)
  read_interruptions = df[SLACK_CONVERSATION_READ_COLUMN_NAME] >= min_interrupt_read_amount
  user_event_interruptions = df[SLACK_USER_EVENT_COLUMN_NAME] >= min_interrupt_send_amount
  if len(df.loc[read_interruptions | user_event_interruptions]) >= min_interrupt_len:
    return True
  return False

def period_interrupted_naive(df):
  read_interruptions = df[SLACK_CONVERSATION_READ_COLUMN_NAME] >= 1
  user_event_interruptions = df[SLACK_USER_EVENT_COLUMN_NAME] >= 1
  if len(df.loc[read_interruptions | user_event_interruptions]) >= 1:
    return True
  return len(df.loc[read_interruptions | user_event_interruptions] >= 1)
    

def deepwork_streak_calculation(df, streak_length=3, streak_func=None):
  # periods are 5 mins each
  REQ_STREAK_LENGTH_PERIODS = streak_length
  INTERRUPTION_PERIOD_LENGTH = 2
  PERIOD_LENGTH = dt.timedelta(minutes=5)
  assert REQ_STREAK_LENGTH_PERIODS > INTERRUPTION_PERIOD_LENGTH
  
  # keeping track of state across looks
  streak = pd.Series(index=df.index) # needs to be pandas._libs.tslibs.timestamps.Timestamp series
  dumb = pd.Series([1, 2, 3])
  streak_start = None
  in_streak = False
  final_period = df.index[-1]
  def get_streak(mini_df):
    nonlocal streak, streak_start, in_streak, final_period
    interrupt_index = mini_df.index[-INTERRUPTION_PERIOD_LENGTH:]
    final_index = mini_df.index[-1][0] + PERIOD_LENGTH*(REQ_STREAK_LENGTH_PERIODS - 1)
    
    if period_interrupted(mini_df.loc[interrupt_index], min_interrupt_len=2):
      if in_streak:
        streak[final_index] = streak_start
        in_streak = False
        streak_start = None
    else:
      if not in_streak:
        if not period_interrupted(mini_df, min_interrupt_len=2): # checking fully uninterrupted time
          in_streak = True
          streak_start = mini_df.index[0][0]
      if (final_index == final_period) & in_streak:
        streak[final_period] = streak_start

  roll(df, REQ_STREAK_LENGTH_PERIODS).apply(get_streak)

  return streak

# CSV defined in gist
test_df = pd.read_csv('sample_interruptions.csv')
test_df.index = pd.DatetimeIndex(test_df['Index'])
deepwork_calculation(test_df)

Index
2020-01-26 06:30:00                    NaN
2020-01-26 06:35:00                    NaN
2020-01-26 06:40:00                    NaN
2020-01-26 06:45:00                    NaN
2020-01-26 06:50:00                    NaN
2020-01-26 06:55:00                    NaN
2020-01-26 07:00:00                    NaN
2020-01-26 07:05:00                    NaN
2020-01-26 07:10:00                    NaN
2020-01-26 07:15:00                    NaN
2020-01-26 07:20:00    2020-01-26 06:30:00
2020-01-26 07:25:00                    NaN
2020-01-26 07:30:00                    NaN
2020-01-26 07:35:00                    NaN
2020-01-26 07:40:00                    NaN
2020-01-26 07:45:00                    NaN
2020-01-26 07:50:00                    NaN
2020-01-26 07:55:00                    NaN
2020-01-26 08:00:00                    NaN
2020-01-26 08:05:00                    NaN
2020-01-26 08:10:00                    NaN
2020-01-26 08:15:00                    NaN
2020-01-26 08:20:00                    NaN
2020-

,Index,slack_conversation_read_count,slack_user_event_count,streak_val,,streak
Index,,,,,,
2020-01-26 06:30:00,1/26/20 6:30,0,0,NaN,NaN,NaN
2020-01-26 06:35:00,1/26/20 6:35,0,0,NaN,NaN,NaN
2020-01-26 06:40:00,1/26/20 6:40,0,0,NaN,NaN,NaN
2020-01-26 06:45:00,1/26/20 6:45,0,0,NaN,NaN,NaN
2020-01-26 06:50:00,1/26/20 6:50,0,0,NaN,NaN,NaN
2020-01-26 06:55:00,1/26/20 6:55,0,0,NaN,NaN,NaN
2020-01-26 07:00:00,1/26/20 7:00,1,0,NaN,NaN,NaN
2020-01-26 07:05:00,1/26/20 7:05,0,0,NaN,NaN,NaN
2020-01-26 07:10:00,1/26/20 7:10,0,0,NaN,NaN,NaN


In [54]:
def collaborative_time_naive(df):
  '''
    df: Pandas Dataframe with Datetime index of 5 minute intervals
    returns: time periods of the df where there have been Slack messages read / sent, or user is not in a meeting 
  '''
  
  return df.loc[(df[SLACK_CONVERSATION_READ_COLUMN_NAME] != 0) \
                | (df[SLACK_USER_EVENT_COLUMN_NAME] != 0) \
                | (df[IN_MEETING_COLUMN_NAME] != 0)
               ].index


In [57]:
collaborative_naive_index = collaborative_time_naive(activity_df)
collaborative_naive_index

Index(['2020-02-02 17:25:00', '2020-02-03 02:25:00', '2020-02-03 02:30:00',
       '2020-02-03 03:10:00', '2020-02-03 17:40:00', '2020-02-03 19:00:00',
       '2020-02-03 19:20:00', '2020-02-03 19:30:00', '2020-02-03 19:40:00',
       '2020-02-03 20:25:00',
       ...
       '2020-02-08 23:10:00', '2020-02-08 23:15:00', '2020-02-08 23:20:00',
       '2020-02-08 23:25:00', '2020-02-08 23:30:00', '2020-02-08 23:35:00',
       '2020-02-08 23:40:00', '2020-02-08 23:45:00', '2020-02-08 23:50:00',
       '2020-02-08 23:55:00'],
      dtype='object', length=376)

In [59]:
len(collaborative_naive_index)/len(activity_df) # % collaborative time

0.16319444444444445

In [ ]:
# filtering out workday hours
